In [5]:
import numpy as np
import cvxpy as cvx

# Problem 1

In [4]:
A = np.array([[1,2,0,1],
              [0,0,3,1],
              [0,3,1,1],
              [2,1,2,5],
              [1,0,3,2]
             ])
c_max = np.array([[100],[100],[100],[100],[100]])
p = np.array([[3],[2],[7],[6]])
p_disc = np.array([[2],[1],[4],[2]])
q = np.array([[4],[10],[5],[10]])

Solving the original problem
$$
\begin{align}
\text{maximize}\ &\sum_{i=1}^{n}r_j(x_j) \\
\text{subject to}\ &x\succcurlyeq 0,\\
&Ax\preccurlyeq c^{max}
\end{align}
$$


In [28]:
x = cvx.Variable((4, 1))
r = cvx.minimum(cvx.multiply(p, x), cvx.multiply(p, q)+cvx.multiply(p_disc, x-q))
prob = cvx.Problem(cvx.Maximize(cvx.sum(r)),
                  [x >= 0,
                  A*x <= c_max])
prob.solve(verbose=True)
print("x:", x.value)
print("r:", r)
print("total:", prob.value)
print("average:", cvx.multiply(r, cvx.inv_pos(x.value)))


ECOS 2.0.4 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  -2.186e+02  -8.578e+02  +8e+02  1e-01  5e-01  1e+00  4e+01    ---    ---    1  1  - |  -  - 
 1  -1.815e+02  -2.916e+02  +2e+02  2e-02  1e-01  4e+00  1e+01  0.8726  1e-01   0  0  0 |  0  0
 2  -1.925e+02  -2.247e+02  +6e+01  6e-03  3e-02  1e+00  3e+00  0.7748  1e-01   0  0  0 |  0  0
 3  -1.917e+02  -1.968e+02  +9e+00  1e-03  5e-03  2e-01  5e-01  0.9415  1e-01   0  0  0 |  0  0
 4  -1.924e+02  -1.932e+02  +1e+00  1e-04  8e-04  3e-02  8e-02  0.8738  2e-02   0  0  0 |  0  0
 5  -1.925e+02  -1.925e+02  +2e-02  2e-06  9e-06  4e-04  9e-04  0.9890  8e-04   1  0  0 |  0  0
 6  -1.925e+02  -1.925e+02  +2e-04  2e-08  1e-07  4e-06  1e-05  0.9890  1e-04   1  0  0 |  0  0
 7  -1.925e+02  -1.925e+02  +2e-06  2e-10  1e-09  5e-08  1e-07  0.9890  1e-04   1  0  0 |  0  0

OPTIMAL (within feastol=1.1e-09, reltol=9.9e-

Solving the LP 
$$
\begin{aligned}
\text{maximize}\qquad &1^\top t\\
\text{subject to}\qquad &x\succcurlyeq 0,\\
    &Ax\preccurlyeq c^{max},\\
    &p_jx_j\ge t_j,\\
    &p_jq_j+p_j^{disc}(x_j-q_j)\ge t_j
\end{aligned}
$$

In [31]:
t = cvx.Variable((4, 1))
x = cvx.Variable((4, 1))
one = np.ones((1, 4))
prob = cvx.Problem(cvx.Maximize(cvx.sum(t)), 
                  [x >= 0,
                  A*x <= c_max,
                  cvx.multiply(p, x) >= t,
                  cvx.multiply(p, q)+cvx.multiply(p_disc, (x-q)) >= t
                  ])
prob.solve(solver='SCS', verbose=True)

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 35, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 8, constraints m = 17
Cones:	linear vars: 17
Setup time: 5.66e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.50e+01  3.34e+01  5.00e-01 -8.56e+03 -2.85e+03  0.00e+00  2.30e-02 
    80| 4.26e-07  8.78e-07  1.30e-07 -1.92e+02 -1.93e+02  7.30e-14  3.14e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: 

192.49996452157424

# Problem 2
(a) Left is convex, but the equality constraint is valid only when both sides are affine. $x == 0, y == 0$  
(b) `square` is convex and is neither nonincreasing nor nondecreasing, so it can only accept affine inputs. Change to $(x+y)^4$.  
(c) $\frac{1}{x}$ is not convex on $\mathbb{R}_{+}$. Need to change constraints to $x > 0, y > 0$.  
(d) `norm` is convex and is neither nonincreasing nor nondecreasing, so it can only accept affine inputs. Change to 
    $$ norm([s, t]) \le 3x+y \\
       \max(x, 1) \le s, \max(y, 2) \le t
    $$
(e) $xy$ is not concave. Change to `x >= inv_pos(y)`  
(f) Cannot divide a convex function with an concave func. Change to `quad_over_lin([x+y], sqrt(y))` 
(g) $x^3$ is not convex in whole domain. Change to `power(x, 3)+power(y, 3)`  
(h) same with (e). change to `x+z <= geomean([y, z-quad_over_lin(z, y)])`

# Problem 3